In [1]:
#Importing packages
from selenium import webdriver
from pymongo import MongoClient
from collections import Counter
import pandas as pd
import json
import twint
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx
import nest_asyncio

In [13]:
driver = webdriver.Chrome()
driver.get('https://www.poynter.org/ifcn-covid-19-misinformation/?covid_countries=48837&covid_rating=0&covid_fact_checkers=0&orderby=views&order=DESC')

In [14]:
client = MongoClient('mongodb://localhost:27017/')

In [15]:
news_list = []
user_list = []
edge_list = []
users = []
counter = 0

In [5]:
#if popup_close = driver.find_element_by_xpath('/html/body/div[9]/div[1]/form/div[3]/span/p/span'):
#   popup_close.click()

In [16]:
for page_num in range(2,6):
    for x in range(0,14):
        button = driver.find_elements_by_link_text('READ MORE')
        button[x].click()
        validity = driver.find_element_by_class_name('entry-title--red').text
        validity = validity[:-1]
        claim = driver.find_element_by_class_name('entry-title').text
        if not claim.strip():
            driver.back()
        
        else:
            news_list.append({'nodetype':"news", 'validity':validity.upper(), 'claim':claim, 'idnum':counter})
            counter = counter + 1
            driver.back()
    print(page_num)
    button = driver.find_element_by_link_text('{}'.format(page_num))
    button.click()

2
3
4
5
6


In [17]:
print(len(news_list))

70


In [18]:
driver.get('https://www.who.int/emergencies/diseases/novel-coronavirus-2019/advice-for-public/myth-busters')
content = driver.find_elements_by_css_selector('h2')

In [19]:
for con in content:
    if 'FACT: ' not in con.text:
        content.remove(con)
    else:
        news_list.append({'nodetype':"news", 'validity':"TRUE", 'claim':con.text.replace('FACT: ',''), 'idnum':counter})
        counter = counter + 1
#print(len(news_list))

In [20]:
#print(news_list)
with open('news_list.json', 'w') as json_file:
    json.dump(news_list, json_file)   

In [21]:
G = nx.Graph()
a = 0
color_map = []
for news in news_list:
    G.add_node(a, n=news)
    color_map.append('blue')
    a = a + 1

In [22]:
for news in news_list:
    nest_asyncio.apply()
    co = twint.Config() 
    co.Search = news['claim']
    co.Limit = 5
    twint.run.Search(co) 
    tlist = co.search_tweet_list
    for x in tlist:
        name = x['username']
        if name in users:
            count = 0
            for x in user_list:
                if x['username'] == name:
                    count = x['idnum']
                    break
            edge_list.append({'type':'nu', 'edge':[count,news['idnum']]})    
        else:
            users.append(name)
            user_list.append({'nodetype':'user', 'username':name, 'idnum':counter})
            edge_list.append({'type':'nu', 'edge':[counter,news['idnum']]})
            counter = counter + 1

CRITICAL:root:twint.run:Twint:Feed:noDatalist index out of range
sleeping for 1.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDatalist index out of range
sleeping for 8.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDatalist index out of range
sleeping for 27.0 secs


KeyboardInterrupt: 

In [ ]:
print(user_list)

In [ ]:
with open('user_list.json', 'w') as json_file:
    json.dump(user_list, json_file)

In [ ]:
for user in user_list:
    G.add_node(a, u=user)
    color_map.append('green')
    a = a + 1

In [ ]:
#print(G.number_of_nodes())
#print(G.nodes[0])
#print(G.number_of_edges())

In [ ]:
for edges in edge_list:
    G.add_edge(edges['edge'][0],edges['edge'][1])

In [ ]:
nx.draw_random(G, node_color=color_map, with_labels=True)
plt.savefig("network.png")
plt.show()

In [ ]:
print(nx.average_clustering(G))

In [ ]:
print(nx.reciprocity(G))